# OER Volcano for IrOx systems

***

# Import Modules | TEMP NEW

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

In [ ]:
import os
print(os.getcwd())
import sys

In [ ]:
%%capture

sys.path.insert(
    0, os.path.join(
        os.environ["PROJ_irox"],
        "workflow"))

sys.path.insert(
    0, os.path.join(
        os.environ["PROJ_irox"],
        "data"))

from an_data_processing import load_df

# #############################################################################
# Python Modules

import pickle

import numpy as np

import plotly.graph_objs as go

# #############################################################################
# My Modules
from oxr_reaction.oxr_rxn import ORR_Free_E_Plot
from oxr_reaction.oxr_plotting_classes.oxr_plot_volcano import Volcano_Plot

# #############################################################################
# Project Data
from proj_data_irox import (
    proj_dir_name,
    smart_format_dict,
    gas_molec_dict,
    scaling_dict_ideal,
    scaling_dict_fitted,
    exp_irox_lim_pot,
    data_dir,
    groupby_props,
    axis_label_font_size,
    axis_tick_labels_font_size,
    oer_systems_to_plot,
    irox_bulk_color_map)

# #############################################################################
# Local Imports
from plotting.my_plotly import (
    my_plotly_plot,
    add_minor_ticks,
    add_duplicate_axes,
    )

# from layout import layout
from layout2 import layout

# Script Inputs

In [ ]:
save_plot = False
plot_exp_traces = True

plot_range = {
    "y": [2., 1.4],
    "x": [1., 2.],
    }

# Read and Process Data Frame

## Read dataframe from file

In [ ]:
df_pourbaix, df_ads, df_surf = load_df(
# tmp = load_df(
    from_file=False,
    root_dir=data_dir,
    data_dir=data_dir,
    file_name="df_master.pickle",
    process_df=True,
    )

df_m = df_ads

In [ ]:
# assert False

# ORR_Free_E_Plot Instance

In [ ]:
ORR_PLT = ORR_Free_E_Plot(
    free_energy_df=None,
    state_title="adsorbate",
    free_e_title="ads_e",
    smart_format=smart_format_dict,
    color_list=None,
    rxn_type="OER")

# Processing Data

In [ ]:
import pandas as pd

In [ ]:
# df_m.head()

In [ ]:
new_index_order = [] + \
    df_m[df_m.bulk_system != "IrO3"].index.tolist() + \
    df_m[df_m.bulk_system == "IrO3"].index.tolist() + \
    []

df_m = df_m.loc[new_index_order]

In [ ]:
index_i = df_m[
    (df_m.bulk_system == "IrO3_rutile-like") & \
    (df_m.facet == "100") & \
    (df_m.coverage_type == "o_covered_2") & \
    (df_m.adsorbate == "o")
    ].iloc[0:].index[0]

# 2.840912 eV
# df_m.loc[274, "ads_e"] = 2.78
# df_m.loc[274, "ads_e"] = 2.838
df_m.loc[index_i, "ads_e"] = 2.838


index_i = df_m[
    (df_m.bulk_system == "IrO3_rutile-like") & \
    (df_m.facet == "110") & \
    (df_m.coverage_type == "o_covered") & \
    (df_m.adsorbate == "o")
    ].iloc[0:].index[0]

# 2.62689
df_m.loc[index_i, "ads_e"] = 2.63

In [ ]:
# assert False

In [ ]:
prop_name_list = [
    'bulk_system',
    # 'coverage',
    'coverage_type',
    'facet',
    'surface_type',
    ]

grouped = df_m.groupby(groupby_props, sort=False)

for i_ind, (name, group) in enumerate(grouped):
    df_i = group
    
    name_i = "_".join(list(name))
    print("name:", name_i)

    if not any([np.isnan(i) for i in df_i.elec_energy.tolist()]):
        if name_i in oer_systems_to_plot:
            ORR_PLT.add_series(
                df_i,
                plot_mode="all",
                overpotential_type="OER",
                property_key_list=prop_name_list,
                add_overpot=False,
                name_i=name_i,
                )

# Experimental IrOx Activity Traces

## Horizontal data traces

In [ ]:
trace_iro3 = go.Scatter(
    x=plot_range["x"],
    y=2 * [exp_irox_lim_pot["10_mA/cm2"]["SrIrO3"]],    
    mode="lines",
    name="lines",
    line={
        "color": irox_bulk_color_map["IrO3"],
        "width": 2.,
        # "dash": "dash",
        "dash": "dot",
        },
    )

trace_iro2 = go.Scatter(
    x=plot_range["x"],
    # y=2 * [exp_irox_lim_pot["10_mA/cm2"]["IrO2(110)"]],
    y=2 * [exp_irox_lim_pot["10_mA/cm2"]["IrO2(110)_R95"]],
    mode="lines",
    name="lines",
    line={
        "color": irox_bulk_color_map["IrO2"],
        "width": 2.,
        "dash": "dot",
        },
    )

# #############################################################################

annot_shared = go.layout.Annotation(

bgcolor="white",
font=go.layout.annotation.Font(
    color="black",
    family=None,
    # size=axis_label_font_size,
    size=axis_tick_labels_font_size,
    ),
    opacity=0.8,
    showarrow=False,
    x=layout.xaxis.range[0],
    xanchor="left",
    # xclick=None,
    xref="x1",
    xshift=0.,
    # y=None,
    yanchor="bottom",
    # yclick=None,
    yref="y1",
    # yshift=30,
    yshift=2,
    )


annotations_exp = [

    go.layout.Annotation(
        # text="SrIrO<sub>3</sub>",
        text="IrO<sub>x</sub>/SrIrO<sub>3</sub> @10 mA/cm<sup>2</sup>",
        y=exp_irox_lim_pot["10_mA/cm2"]["SrIrO3"],
        name="exp_lim_pot_SrIrO3",
        **annot_shared.to_plotly_json()),

    go.layout.Annotation(
        text="R-IrO<sub>2</sub> (110) @10 mA/cm<sup>2</sup>",
        # y=exp_irox_lim_pot["10_mA/cm2"]["IrO2(110)"],
        y=exp_irox_lim_pot["10_mA/cm2"]["IrO2(110)_R95"],
        
        name="exp_lim_pot_IrO2_110",
        **annot_shared.to_plotly_json()),

    ]


layout.update(
    annotations=list(layout.annotations) + list(annotations_exp),
    overwrite=True)

tmp = 42

# Volcano Plot

In [ ]:
VP = Volcano_Plot(
    ORR_PLT,
    x_ax_species="o-oh",  # 'o-oh' or 'oh'
    smart_format_dict=smart_format_dict,
    plot_range=plot_range,
    )

VP.create_volcano_relations_plot()

volcano_legs_data = VP.create_volcano_lines(
    gas_molec_dict=gas_molec_dict,
    scaling_dict=scaling_dict_ideal,
    plot_all_legs=False,
    plot_min_max_legs=True,
    trace_priority="bottom",  # 'top' or 'bottom'
    )

volcano_legs_data_tmp = VP.create_volcano_lines(
    gas_molec_dict=gas_molec_dict,
    scaling_dict=scaling_dict_fitted,
    plot_all_legs=False,
    plot_min_max_legs=True,
    trace_priority="bottom",  # 'top' or 'bottom'
    legs_to_plot=[
        # "o2_to_ooh",
        "ooh_to_o",
        "o_to_oh",
        # "oh_to_h2o",
        ],
    line_color="grey"
    )

# data = volcano_legs_data + volcano_legs_data_tmp + VP.data_points
data = volcano_legs_data_tmp + volcano_legs_data + VP.data_points

if plot_exp_traces:
    data.insert(0, trace_iro3)
    data.insert(0, trace_iro2)

In [ ]:
fig = go.Figure(data=data, layout=layout)

my_plotly_plot(
    figure=fig,
    plot_name="out_plot_02_large")

# TEMP | Changing line type of volcano

In [ ]:
shared_axis_props = dict(ticklen=3)

ticks_props_new_x = dict(
    dtick=0.1,
    **shared_axis_props)
ticks_props_new_y = dict(
    dtick=0.05,
    **shared_axis_props)


add_duplicate_axes(
    fig, axis_type='x',
    axis_data=ticks_props_new_x,
    tmp_define_both_axis_types=False,
    )

add_duplicate_axes(
    fig, axis_type='y',
    axis_data=ticks_props_new_y,
    tmp_define_both_axis_types=False,
    )

my_plotly_plot(
    figure=fig,
    plot_name="pl_irox_volcano_plotly_default_ooh",
    write_html=True,
    write_png=False,
    png_scale=6.0,
    write_pdf=True,
    write_svg=False,
    try_orca_write=True,
    )

In [ ]:
fig.show()

In [ ]:
# fig.show()

In [ ]:
# import copy

# tmp = copy.deepcopy(fig.layout)
# tmp.annotations = None

# tmp

# Adding Kinetic Volcano Traces

In [ ]:
# #############################################################################
path_i = os.path.join(
    "out_data",
    "kinetic_volcano_trace.pickle")
with open(path_i, "rb") as fle:
    data_kin_volc = pickle.load(fle)
# #############################################################################

# #############################################################################
path_i = os.path.join(
    "out_data",
    "df_10mA.pickle")
with open(path_i, "rb") as fle:
    df_10mA = pickle.load(fle)
# #############################################################################

In [ ]:
df = df_10mA
trace_kin_10mA = go.Scatter(
    x=df["descriptor"],
    y=df["U_lim"],
    mode="lines",
    name="temp_4348",
    line=dict(
        # color="#1ee148",
        color="#3e9bf2", 
        width=2.,
        dash="dot",
        ),
    )

fig.add_scatter(**trace_kin_10mA.to_plotly_json())

tmp = 42

In [ ]:
data_tmp = fig.data

data = list(data_tmp)[-1:] + list(data_tmp)[0:-1]
fig.data = data

In [ ]:
# assert False

In [ ]:
tmp = my_plotly_plot(
    figure=fig,
    plot_name="pl_irox_volcano_plotly_default_ooh__w_kinetic_volc",
    write_html=True,
    write_pdf=True,
    write_svg=False,
    try_orca_write=True)

fig.show()

In [ ]:
# Pickling data ######################################################
import os; import pickle
directory = "out_data"
if not os.path.exists(directory): os.makedirs(directory)
with open(os.path.join(directory, "oer_volcano_trace.pickle"), "wb") as fle:
    pickle.dump(fig, fle)
# #####################################################################

In [ ]:
# for i_cnt, trace in enumerate(fig.data):
#     tmp = 42
#     print(trace.name)
#     if trace.name == "temp_4348":
#         print(trace)
#         print("ISDFIJISDJFIJSDI")

# fig.data[-1:]
# print(len(fig.data))
# print(len(fig.data))
# data_0 = list(fig.data)
# data_0.insert(0, trace_kin_10mA)
# fig.data = data_0

# data_tmp = fig.data

# # print(len(fig.data))
# # len(list(data_tmp)[0:-1])
# # list(data_tmp)

# for i_cnt, trace in enumerate(fig.data):
#     tmp = 42
#     print(trace.name)
#     if trace.name == "temp_4348":
#         print(trace)
#         print("ISDFIJISDJFIJSDI")

# for trace_i in data_kin_volc:
#     data_dict = trace_i.to_plotly_json()
#     fig.add_scatter(**data_dict)

In [ ]:
## for trace in fig.data:
#     tmp = 42
    
# #     print(trace.name)
    
#     if trace.name == "activity volcano":
#         trace_tmp = trace

# trace_tmp.line.dash = "7px,2px,7px,2px"

In [ ]:
for i in ORR_PLT.series_list:
    print(
        i.name_i,
        ":",
        i.limiting_step
        )

# ORR_PLT.series_list[0].limiting_step

In [ ]:
i.name_i


In [ ]:
df_m.name_i_2.unique().tolist()

In [ ]:
# df_m.head()

In [ ]:
## # df_m = pd.concat([
# #     df_m[df_m.bulk_system != "IrO3"],
# #     df_m[df_m.bulk_system == "IrO3"],
# #     ])


# # #####################################
# prop_name_list = [
#     'bulk_system',
#     # 'coverage',
#     'coverage_type',
#     'facet',
#     'surface_type']
# grouped = df_m.groupby(groupby_props, sort=False)
# for i_ind, (name, group) in enumerate(grouped):
#     df_i = group
#     name_i = "_".join(list(name))
#     print("name:", name_i)